In [ ]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
import aot

In [ ]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)  

settings_root_path = base_dir / core_settings["paths"]["settings_path"] 
video_db_path = base_dir / 'data/videos/database_originals.tsv'
video_db = pd.read_csv(video_db_path, sep='\t')

In [ ]:
#videos_names = video_db['video_name'].values
#traverse the db and get the videos names that grade j or NA
videos_names = []
for i in range(1,len(video_db)):
    if video_db['grade'][i] != 'k' and video_db['grade'][i] != 'l':   #video_db['grade'][i] == 'j' or video_db['grade'][i] == 'NA':
        videos_names.append(video_db['video_name'][i])
print("videos number: ", len(videos_names))

origin_videos_number = len(videos_names)

subject_number = 20
session_number = 8
run_number = 10
blank_number_per_run = core_settings['various']['blank_number_per_run']#only for main run
unique_video_number_per_session = core_settings['various']['unique_video_number_per_session']
video_number_per_run_main = int((unique_video_number_per_session*4) / run_number)
video_number_per_run_grading = int(origin_videos_number / run_number) #only 1 subject and 1 session for grading

print("videos names: ", videos_names)
print("subject number: ", subject_number)
print("session number: ", session_number)
print("run number: ", run_number)
print("unique video number per session: ", unique_video_number_per_session)



In [ ]:
def make_settings_for_one_session(subject_number,session_number,original_video_names,exp_tupe = "eyetracking"):#input videos that before selection
    def make_settings_for_one_run(subject_number,session_number,run_number,video_names,exp_type = "eyetracking"):#input videos that after selection
        #i is run number
        settings = copy.deepcopy(stimuli_settings_temp)
        settings['stimuli']['movie_files'] = video_names
        if exp_type == "eyetracking":
            #settings['stimuli']['movie_files'] = settings['stimuli']['movie_files'] + ["blank" for i in range(blank_number_per_run)]
            #random.shuffle(settings['stimuli']['movie_files'])
            
            #uniformly distribute the blanks into movies
            movie_number_per_blank = int(len(settings['stimuli']['movie_files']) / blank_number_per_run)
            for i in range(blank_number_per_run):
                perturb = random.randint(-1,1)
                #settings['stimuli']['movie_files'].insert((i+1)*movie_number_per_blank+i,"blank")
                settings['stimuli']['movie_files'].insert((i+1)*movie_number_per_blank+i+perturb,"blank")


        #save settings
        with open(
            str(settings_root_path)
            + "/"
            + exp_type
            + "/experiment_settings_" 
            + "sub_"
            + subject_number.zfill(2)
            + "_ses_"
            + session_number.zfill(2)
            + "_run_"
            + run_number.zfill(2)
            + ".yml",
            "w",
        ) as outfile:
            print(
            str(settings_root_path)
            + "/"
            + exp_type
            + "/experiment_settings_"
            + "sub_"
            + subject_number.zfill(2)
            + "_ses_"
            + session_number.zfill(2)
            + "_run_"
            + run_number.zfill(2)
            + ".yml",
            "w",
            )
            print(settings)
            print(" ")
            yaml.dump(settings, outfile, default_flow_style=False)
        return settings
        
        

    original_video_names = copy.deepcopy(original_video_names)
    random.shuffle(original_video_names)
    #get first {video_number_per_session} videos         for sessions in main/eyetracking experiment 
    if exp_tupe == "eyetracking": 
        original_video_names = original_video_names[:unique_video_number_per_session]
    #else:#for grading and labeling we use all the videos in the only session
    #   original_video_names = original_video_names

    

    resampled_video_names = ["S_"+name for name in original_video_names]#for grading and labeling
    reversed_video_names = ["R_"+name for name in resampled_video_names]
    total_video_names = copy.deepcopy(resampled_video_names) + copy.deepcopy(reversed_video_names)
    doubled_total_video_names = copy.deepcopy(total_video_names) + copy.deepcopy(total_video_names)#for main exp
    random.shuffle(total_video_names)
    random.shuffle(doubled_total_video_names)

    for i in range(1,run_number+1):
        if i == run_number:
            #make settings for main/eyetracking run
            if exp_tupe == "eyetracking":
                video_names = doubled_total_video_names[(i-1)*video_number_per_run_main:]
                make_settings_for_one_run(subject_number,session_number,str(i),video_names,exp_type = 'eyetracking')
            else:
            #make settings for grading and labeling run
                video_names = resampled_video_names[(i-1)*video_number_per_run_grading:]
                make_settings_for_one_run(subject_number,session_number,str(i),video_names,exp_type = exp_tupe)

        else: 
            if exp_tupe == "eyetracking":  
                #make settings for main/eyetracking run
                video_names = doubled_total_video_names[(i-1)*video_number_per_run_main:i*video_number_per_run_main]
                make_settings_for_one_run(subject_number,session_number,str(i),video_names,exp_type = 'eyetracking')
            else:
            #make settings for grading and labeling run
                video_names = resampled_video_names[(i-1)*video_number_per_run_grading:i*video_number_per_run_grading]
                make_settings_for_one_run(subject_number,session_number,str(i),video_names,exp_type = exp_tupe)


In [ ]:
def make_settings_for_one_subject(subject_number,original_video_names,exp_tupe = "eyetracking"):
    if exp_tupe == "eyetracking":
        for i in range(1,session_number+1):
            make_settings_for_one_session(subject_number,str(i),original_video_names)
    #else:#for grading and labeling there is only one session 
    #    make_settings_for_one_session(subject_number,"1",original_video_names,exp_tupe = exp_tupe)

def make_settings_for_all_subjects(original_video_names,exp_tupe = "eyetracking"):
    if exp_tupe == "eyetracking":
        for i in range(1,subject_number+1):
            make_settings_for_one_subject(str(i),original_video_names,exp_tupe = exp_tupe)
    #else:#for grading and labeling there is only one subject
    #    make_settings_for_one_subject("1",original_video_names,exp_tupe = exp_tupe)

In [ ]:
make_settings_for_all_subjects(videos_names,exp_tupe = "eyetracking")